In [1]:
import numpy as np
import pandas as pd
import random
from itertools import combinations, permutations

In [2]:
models=['bert','gpt2','roberta','electra','xlm','lstm','rnn','bigram','trigram']

sheet=pd.read_excel('cont_stim_pilot_all.xlsx')

models1=list(sheet['m1'])
models2=list(sheet['m2'])
natural=list(sheet['N'])
sents1=list(sheet['S1'])
sents2=list(sheet['S2'])
rank1=list(sheet['p_N_given_m1_decile'])
rank2=list(sheet['p_N_given_m2_decile'])

unique_models1=list(set(models1))
unique_models2=list(set(models2))

In [3]:

poss_stim=[]

for i in range(30):
    
    trip_ind=int(i/3)
    
    type_ind=i%3

    models1_sub=models1[trip_ind::10]
    models2_sub=models2[trip_ind::10]
    
    natural_sub=natural[trip_ind::10]
    sents1_sub=sents1[trip_ind::10]
    sents2_sub=sents2[trip_ind::10]
    
    for j in range(len(models1_sub)):
        
        m1=models1_sub[j]
        m2=models2_sub[j]
        
        n=natural_sub[j]
        s1=sents1_sub[j]
        s2=sents2_sub[j]
       
        if type_ind==0:
            type1='-'.join([m1,m2,'N','S1'])
            trial=[type1,m1,m2,'N','S1',n,s1]
        elif type_ind==1:
            type1='-'.join([m1,m2,'N','S2'])
            trial=[type1,m1,m2,'N','S2',n,s2]
        elif type_ind==2:
            type1='-'.join([m1,m2,'S1','S2'])
            trial=[type1,m1,m2,'S1','S2',s1,s2]
            
        poss_stim.append(trial)

In [4]:
sheet=pd.read_excel('cont_stim_pilot_natural.xlsx')

models1=list(sheet['m1'])
models2=list(sheet['m2'])
sents1=list(sheet['S1'])
sents2=list(sheet['S2'])

for i in range(len(sheet)):
    
    poss_stim.append([sheet['m1'][i]+'-'+sheet['m2'][i]+'-N1-N2',sheet['m1'][i],sheet['m2'][i],'N1','N2',sheet['S1'][i],sheet['S2'][i]])


In [5]:
used_sents=[p[5] for p in poss_stim]+[p[6] for p in poss_stim]
all_sents=open('sents_reddit_natural_June2021_filtered.txt','r')
all_sents=all_sents.read().split('\n')
all_sents=list(set(all_sents)-set(used_sents))
random.shuffle(all_sents)
add_sents=all_sents[:300]

In [6]:
sent_ind=0
for i in range(90):
        s1=add_sents[sent_ind]
        s2=add_sents[sent_ind+1]
        sent_ind+=2
        poss_stim.append(['all-all-R1-R2','all','all','R1','R2',s1,s2])

        
for i in range(120):
        s1=add_sents[sent_ind]
        w=s1.split(' ')
        random.shuffle(w)
        s2=' '.join(w)
        sent_ind+=1
        poss_stim.append(['all-all-C1-C2','all','all','C1','C2',s1,s2])

In [7]:
len(poss_stim)/10


165.0

In [8]:
random.shuffle(poss_stim)

type_strs=list(set([p[0] for p in poss_stim]))


In [9]:
done=[]

sub_all=[]

for i in range(10):
    
    sub=[]
    
    for type1 in type_strs:
        
        poss_trials=[p for p in poss_stim if p[0]==type1]

        if type1[-2:]=='C2':
            
            ind=0
            for trial in poss_trials:

                if trial not in done:
                    sub.append(trial)
                    done.append(trial)
                    ind+=1
                    if ind==12:
                        break
                        
        elif type1[-2:]=='R2':
            
            ind=0
            for trial in poss_trials:

                if trial not in done:
                    sub.append(trial)
                    done.append(trial)
                    ind+=1
                    if ind==9:
                        break
            
        else:
            
            for trial in poss_trials:

                if trial not in done:
                    sub.append(trial)
                    done.append(trial)
                    break
                
    sub_all.append(sub)

In [22]:
sub_sets_all=[]
for sub in sub_all:
    
    sub_sets=[]
    
    for i in range(10):
    
        random.shuffle(sub)

        sub2=[]
        for si,s in enumerate(sub):

            r=random.randint(0,1)

            if r==0:

                t=[s[0],s[1],s[2],s[3],s[4],s[5],s[6],0]

            elif r==1:

                t=[s[0],s[2],s[1],s[4],s[3],s[6],s[5],1]

            sub2.append(t)

        sub_sets.append(sub2)
        
    sub_sets_all.append(sub_sets)

    

In [23]:
frame=pd.DataFrame()

frame['randomise_blocks']=['']*167
frame['randomise_trials']=['']*167
frame['display']=['Instructions']+['Trial']*165+['End']
frame['ANSWER']=['']*167
frame['Progress']=['']+[1 for i in range(165)]+['']

for i in range(10):

    for j in range(10):

#         frame['subject_set '+str(i+1)+'_'+str(j+1)]=['']+[i for j in range(1,166)]+['']
        frame['trial_set '+str(i+1)+'_'+str(j+1)]=['']+list(np.arange(1,166))+['']
        frame['trial_type_set '+str(i+1)+'_'+str(j+1)]=['']+[s[0] for s in sub_sets_all[i][j]]+['']

        frame['sentence1_model_set '+str(i+1)+'_'+str(j+1)]=['']+[s[1] for s in sub_sets_all[i][j]]+['']
        frame['sentence2_model_set '+str(i+1)+'_'+str(j+1)]=['']+[s[2] for s in sub_sets_all[i][j]]+['']
        frame['sentence1_type_set '+str(i+1)+'_'+str(j+1)]=['']+[s[3] for s in sub_sets_all[i][j]]+['']
        frame['sentence2_type_set '+str(i+1)+'_'+str(j+1)]=['']+[s[4] for s in sub_sets_all[i][j]]+['']
        frame['sentence1_set '+str(i+1)+'_'+str(j+1)]=['']+[s[5] for s in sub_sets_all[i][j]]+['']
        frame['sentence2_set '+str(i+1)+'_'+str(j+1)]=['']+[s[6] for s in sub_sets_all[i][j]]+['']
        frame['LR_flipped_set '+str(i+1)+'_'+str(j+1)]=['']+[s[7] for s in sub_sets_all[i][j]]+['']


In [25]:
frame.to_excel('spreadsheet_contstim_confidence_June2021.xlsx')